# Download Individual Feature Rasters

Downloads each feature band as a separate GeoTIFF, aligned to the same grid as
the main stacked raster (`ouaga_aligned_stack.tif`). Useful for inspecting
individual layers in QGIS or re-downloading a single band without re-running
the full pipeline.

All bands are downloaded with identical `scale`, `crs`, and `region` parameters,
so the output rasters are pixel-aligned by construction.

**Output:** `data/processed/{band_name}.tif` (one file per band)

In [ ]:
import sys
import warnings

import ee
import geemap
import numpy as np
import rasterio

from pathlib import Path

sys.path.insert(0, "..")
from src.data import load_config
from src.pipeline import compute_all_features, load_aoi

warnings.filterwarnings("ignore")

In [ ]:
# =============================================================================
# INITIALIZE
# =============================================================================
config = load_config("../config/processing.yaml")

try:
    ee.Initialize(project=config["ee_project"])
    print("Earth Engine initialized successfully")
except Exception:
    ee.Authenticate()
    ee.Initialize(project=config["ee_project"])
    print("Earth Engine initialized successfully")

In [ ]:
# =============================================================================
# COMPUTE FEATURES (reuses the main pipeline)
# =============================================================================
aoi = load_aoi(config)
layers = compute_all_features(aoi, config)

print(f"Computed {len(layers)} layers: {list(layers.keys())}")

In [ ]:
# =============================================================================
# SELECT BANDS TO DOWNLOAD
# =============================================================================
# Edit this list to download only specific bands, or leave as-is for all.
bands_to_download = config["band_names"]

# Examples:
# bands_to_download = ["LST", "NDVI"]
# bands_to_download = ["distance_to_water", "distance_to_roads"]

print(f"Will download: {bands_to_download}")

In [ ]:
# =============================================================================
# DOWNLOAD INDIVIDUAL BANDS
# =============================================================================
output_dir = config["data_dir"]

# Use identical export params as the main pipeline to guarantee alignment
export_params = dict(
    region=aoi,
    scale=config["target_scale"],
    crs=config["target_crs"],
)

downloaded = []
for band_name in bands_to_download:
    out_path = output_dir / f"{band_name}.tif"
    print(f"\nDownloading {band_name} -> {out_path.relative_to(config['data_dir'].parent.parent)}")

    # Apply same transforms as stack_layers(): clip to AOI, cast to Float32
    band_image = layers[band_name].clip(aoi).toFloat()

    geemap.download_ee_image(
        image=band_image,
        filename=str(out_path),
        **export_params,
    )
    downloaded.append(out_path)
    print(f"  Done ({out_path.stat().st_size / 1e6:.1f} MB)")

print(f"\nDownloaded {len(downloaded)} bands to {output_dir}")

In [ ]:
# =============================================================================
# VERIFY ALIGNMENT
# =============================================================================
print("Checking alignment across downloaded bands...\n")

shapes, crses, bounds_list = [], [], []
for path in downloaded:
    with rasterio.open(path) as src:
        shapes.append(src.shape)
        crses.append(str(src.crs))
        bounds_list.append(src.bounds)
        print(f"  {path.stem:.<25} shape={src.shape}  crs={src.crs}  res={src.res}")

all_aligned = len(set(shapes)) == 1 and len(set(crses)) == 1 and len(set(bounds_list)) == 1
print(f"\nAll bands aligned: {all_aligned}")

# Compare against stacked raster if it exists
stack_path = config["raster_path"]
if stack_path.exists() and downloaded:
    print(f"\nComparing against stacked raster ({stack_path.name})...")

    with rasterio.open(stack_path) as ref:
        ref_shape = ref.shape
        ref_bounds = ref.bounds

        for path in downloaded:
            band_name = path.stem
            band_idx = config["band_index"][band_name]  # 1-indexed
            ref_data = ref.read(band_idx)

            with rasterio.open(path) as src:
                test_data = src.read(1)

            if ref_shape != test_data.shape:
                print(f"  {band_name}: SHAPE MISMATCH ref={ref_shape} band={test_data.shape}")
                continue

            valid = ~np.isnan(ref_data) & ~np.isnan(test_data)
            if valid.sum() == 0:
                print(f"  {band_name}: no overlapping valid pixels")
                continue

            max_diff = np.abs(ref_data[valid] - test_data[valid]).max()
            match = np.allclose(ref_data[valid], test_data[valid], atol=1e-6)
            print(f"  {band_name:.<25} max_diff={max_diff:.6f}  exact_match={match}")